In [1]:
import { load } from "dotenv";
const env = await load();

const process = { env };
// process.env;

In [2]:
import { ChatMessageHistory } from "langchain/memory";
import {
  ChatPromptTemplate,
  MessagesPlaceholder,
} from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { RunnableWithMessageHistory, RunnableSequence } from "@langchain/core/runnables";
import { StringOutputParser } from "@langchain/core/output_parsers";

const chatHistory = new ChatMessageHistory();

// console.log(messages);
const chatModel = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

const prompt = ChatPromptTemplate.fromMessages([
  ["system", "你是一个机器人助手，请尽你所能回答用户的问题。"],
  new MessagesPlaceholder("chat_history"),
  ["human", "{humanText}"],
]);

const outputParse = new StringOutputParser();

// const chain = prompt.pipe(chatModel).pipe(outputParse);
const chain = RunnableSequence.from([
  (input) => {
    // console.log('input---', input);
    return input
  },
  prompt,
  (input) => {
    console.log('prompt---', input.messages);
    return input
  },
  chatModel, 
  outputParse
]);

const chatChain = new RunnableWithMessageHistory({
  runnable: chain,
  getMessageHistory: () => chatHistory,
  inputMessagesKey: "humanText",
  historyMessagesKey: "chat_history",
});


In [3]:
const rst1 = await chatChain.invoke({ humanText: "我昨天晚上去了北京，很开心" }, { configurable: { sessionId: "none" } });
// console.log(rst1 + '\n');
const rst2 = await chatChain.invoke({ humanText: "去爬了长城" }, { configurable: { sessionId: "none" } });
// console.log(rst2 + '\n');
const rst3 = await chatChain.invoke({ humanText: "我计划今年冬天的时候再去一次" }, { configurable: { sessionId: "none" } });
const rst4 = await chatChain.invoke({ humanText: "你知道我计划去哪个城市吗" }, { configurable: { sessionId: "none" } });
console.log(rst4);
// console.log(rst3 + '\n');
// const rst4 = await chatChain.invoke({ humanText: "我想去看升旗" }, { configurable: { sessionId: "none" } });
// console.log(rst4 + '\n');
// const rst5 = await chatChain.invoke({ humanText: "会不会很冷" }, { configurable: { sessionId: "none" } });
// console.log(rst5 + '\n');
// const rst6 = await chatChain.invoke({ humanText: "人会不会很多" }, { configurable: { sessionId: "none" } });
// console.log(rst6 + '\n');
// const rst7 = await chatChain.invoke({ humanText: "从深圳过去机票会不会很贵" }, { configurable: { sessionId: "none" } });
// console.log(rst7 + '\n');

prompt--- [
  SystemMessage {
    "content": "你是一个机器人助手，请尽你所能回答用户的问题。",
    "additional_kwargs": {},
    "response_metadata": {}
  },
  HumanMessage {
    "content": "我昨天晚上去了北京，很开心",
    "additional_kwargs": {},
    "response_metadata": {}
  }
]
prompt--- [
  SystemMessage {
    "content": "你是一个机器人助手，请尽你所能回答用户的问题。",
    "additional_kwargs": {},
    "response_metadata": {}
  },
  HumanMessage {
    "content": "我昨天晚上去了北京，很开心",
    "additional_kwargs": {},
    "response_metadata": {}
  },
  AIMessage {
    "content": "很高兴听到您昨天晚上去了北京并且感到开心。北京是一个历史悠久、文化丰富的城市，有很多值得一游的景点和活动。如果您愿意分享更多关于您的北京之行的经历，我会尽力为您提供帮助和建议。",
    "additional_kwargs": {},
    "response_metadata": {},
    "tool_calls": [],
    "invalid_tool_calls": []
  },
  HumanMessage {
    "content": "去爬了长城",
    "additional_kwargs": {},
    "response_metadata": {}
  }
]
prompt--- [
  SystemMessage {
    "content": "你是一个机器人助手，请尽你所能回答用户的问题。",
    "additional_kwargs": {},
    "response_metadata": {}
  },
  HumanMessage {
    "content": "我昨天

In [4]:
import { BufferMemory } from "langchain/memory";
import { ConversationChain } from "langchain/chains";
import {
  ChatPromptTemplate,
} from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { StringOutputParser } from "@langchain/core/output_parsers";
import { RunnableWithMessageHistory, RunnableSequence } from "@langchain/core/runnables";

// console.log(messages);
const llm = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

// const prompt = ChatPromptTemplate.fromMessages([
//   ["system", "你是一个机器人助手，请尽你所能回答用户的问题。"],
//   ["human", "{humanText}"],
// ]);

const outputParser = new StringOutputParser();

const memory = new BufferMemory();

const chain = new ConversationChain({ llm, memory, verbose: false })
// const chatChain = chain.pipe(outputParser);
const chatChain = RunnableSequence.from([chain, (res) => res.response ]);

In [5]:
const rst1 = await chatChain.invoke({ input: "我昨天晚上去了北京，很开心" });
console.log(rst1);
// const rst2 = await chatChain.invoke({ humanText: "去爬了长城" });
// console.log(rst2);
const rst3 = await chatChain.invoke({ input: "我计划今年冬天的时候再去一次" });
console.log(rst3);
const rst4 = await chatChain.invoke({ input: "你知道我说的是去哪个城市吗" });
console.log(rst4);
// const rst5 = await chatChain.invoke({ humanText: "会不会很冷" });
// console.log(rst5 + '\n');
// const rst6 = await chatChain.invoke({ humanText: "人会不会很多" });
// console.log(rst6 + '\n');
// const rst7 = await chatChain.invoke({ humanText: "从深圳过去机票会不会很贵" });
// console.log(rst7 + '\n');
// await chatChain.invoke({ humanText: "我前面说的话你不记得了？" });

哦，北京是一个非常有趣的城市！您昨天晚上都做了些什么呢？有没有参观什么特别的地方？
哦，那太棒了！北京有很多值得一游的地方。您去年冬天去参观了哪些地方吗？
哦，我明白了，您指的是北京市。北京市是中国的首都，拥有丰富的历史文化遗产和现代化都市风貌。如果您去年冬天去北京旅游，您可能参观了故宫博物院、天安门广场、长城等著名景点。祝您去年北京的旅行愉快！
